In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import math
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
class FeatureDataset(Dataset):
    def __init__(self, file_name):
        # read csv file and load row data into input and output
        raw_data = pd.read_csv(file_name)
        inputX = raw_data.iloc[0:len(raw_data), 0:15].values
        outputY = raw_data.iloc[0:len(raw_data), 15].values
        
        #convert to torch tensors
        self.data_X = torch.tensor(inputX,dtype=torch.float32,device=device)
        self.data_Y = torch.tensor(outputY,device=device)
    
    def __len__(self):
        return len(self.data_Y)
    
    def __getitem__(self,idx):
        return self.data_X[idx], self.data_Y[idx]
    
dataSet = FeatureDataset("F:\githubClone\TCN\m100_for_model.csv")

lenTrainingPercen = 80
lenTestPercen = 20
numTraining = math.ceil((len(dataSet)/100)*lenTrainingPercen)
numTest = len(dataSet)-numTraining
print(numTraining)
print(numTest)
train, val = torch.utils.data.random_split(dataSet, [numTraining, numTest])  # spilt the data to training set as well as test set
len(train.indices)

206317
51579


206317

In [3]:
# DATA = pd.read_csv("F:\m100_for_model.csv")
# k = DATA.iloc[0:len(DATA),15]
# k
# DATA_pureInput = DATA.drop(['energy','aircraft'], axis = 1).values
# Input_tensor = torch.tensor(DATA_pureInput)
# DATA_output = DATA['energy'].values
# Label_tensor = torch.tensor(DATA_output)
# Label_tensor
# x,y = torch.utils.data.random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))
# print(x.indices)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset = train,
                                           batch_size = 2048, 
                                           shuffle = True)
val_loader = torch.utils.data.DataLoader(dataset = val, 
                                         batch_size = 10000,  # for batch size default is 1
                                         shuffle = True) 
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)  # X = example_data, Y = example_targets
print(example_targets.shape)


torch.Size([2048])


In [5]:
class ownMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ownMLP, self).__init__()
        self.classifier = nn.Sequential(
            #nn.Flatten(),
            nn.Linear(input_dim,44), #1, # 1st para is input size after flatten, output is the number of neurons in hidden layers after input layer
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 2
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 3
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 4
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 5
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 6
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 7
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 8
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 9
            nn.ReLU(inplace=True),   
            nn.Linear(44,44), # 10
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 11
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 12
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 13
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 14
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 15
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 16
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 17
            nn.ReLU(inplace=True),    
            nn.Linear(44,44), # 18
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 19
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 20
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 21
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 22
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 23
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 24
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 25
            nn.ReLU(inplace=True),   
            nn.Linear(44,44), # 26
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 27
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 28
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 29
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 30
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 31
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 32
            nn.ReLU(inplace=True),
            nn.Linear(44,44), # 33
            nn.ReLU(inplace=True),        
            nn.Linear(44,44), # 34
            nn.ReLU(inplace=True),
            nn.Linear(44,output_dim), # 35
            nn.ReLU(inplace=True),             
        )

    def forward(self, x: torch.Tensor):
        x = self.classifier(x)
        return x
    
model = ownMLP(15,1)
model.to(device)
criterion = nn.MSELoss()  # NOTE: MESLoss() is used for regression, while CrossEntroyLoss() is used for classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)  

In [19]:
mseList = []

for epoch in range(1):
    acc = 0
    valid_acc = 0
    for i, (inputData, outputData) in enumerate(train_loader):  # sweep through the entire training set
        outputData = outputData.unsqueeze(1)
        model.train()
        inData = inputData.to(device)
        outData = outputData.to(device)
        # Forward pass
        outputs = model(inData)
        loss = criterion(outputs, outData.float())
        # Backward and optimize, # another word for optimize is also called move the next step/going to next best position
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc += loss*len(inputData)
    acc = acc/numTraining
    #print(acc)
    # After go through entire training set, now we can use the knowledge we learning, to practice with the test set
    for i, (inputData, outputData) in enumerate(val_loader):
        outputData = outputData.unsqueeze(1)
        model.eval()
        inputData = inputData.to(device)
        outputData = outputData.to(device)
        # Forward pass
        outputs = model(inputData)
        #print(outputData.shape)
        val_loss = criterion(outputs, outputData)  # this validation loss is already the MSE loss for every sample points
        valid_acc += val_loss*len(inputData)
    valid_acc = valid_acc/numTest
        #mseList.append(val_loss)


torch.Size([10000, 1])
0
torch.Size([10000, 1])
1
torch.Size([10000, 1])
2
torch.Size([10000, 1])
3
torch.Size([10000, 1])
4
torch.Size([10000, 1])
5
torch.Size([4474, 1])
6


In [7]:
for i, (inputData, outputData) in enumerate(train_loader):
    print(inputData.shape)
    break

torch.Size([2048, 15])
